In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import scipy.stats as stats

from IPython.display import Image

from sklearn.preprocessing import LabelEncoder 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# reading the CSV
items_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
item_categories_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
sales_train_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
shops_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')

test_df = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
sample_submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')

Printing the data for train

In [ ]:
#items
print(items_df.head())
print(items_df.columns)
print(items_df.dtypes)
print(items_df.shape)


In [ ]:
# item_categories
print(item_categories_df.head())
print(item_categories_df.columns)
print(item_categories_df.dtypes)
print(item_categories_df.shape)

In [ ]:
#sales
print(sales_train_df.head())
print(sales_train_df.columns)
print(sales_train_df.dtypes)
print(sales_train_df.shape)

In [ ]:
# shops
print(shops_df.head())
print(shops_df.columns)
print(shops_df.dtypes)
print(shops_df.shape)

printing test set

In [ ]:
# shops
print(test_df.head())
print(test_df.columns)
print(test_df.dtypes)
print(test_df.shape)

# The scheme of database

In [ ]:
Image("../input/database/database.jpg")

**What should I do?**Data preprocessing:

* 1) check the missing values
* 2) drop any columns
* 3) see the categorical values
* 4) normalization the data
* 5) feature scalling

1. Check the missing values

In [ ]:
# printing the count of Nan values
print("#sales_train_df:")
print(sales_train_df.isna().sum().sort_values(ascending=False))
print('')
print("#items_df:")
print(items_df.isna().sum().sort_values(ascending=False))

Ok. I have no NaN values. It's a perfect.

2. drop any columns
For these part, I must understand relationship between columns

In [ ]:
# Here I can initialize functions which can be used for plotting
def plot_plt_scatter(x, y, ymin=10000):
    #plotting the scatter
    plt.figure(figsize=(40, 30))
    plt.scatter(x, y)
    if ymin != False:
        plt.ylim(0, ymin)
    plt.title('')
    plt.show()

    
def plot_plt_histogram(set_x, bins, grid=False, height=70, width=100,
                       need_xlim=False, need_ylim=False, xlim=1000, ylim=1000):
    # plotting the histogram
    plt.figure(figsize=(width, height))
    plt.hist(set_x, bins=bins)
    if need_xlim == True:
        plt.xlim(0, xlim)
    if need_ylim == True:
        plt.ylim(0, ylim)
    plt.title("Histogram")
    plt.grid(grid)
    plt.show()

At first, let watch to shops_df first words. 

In [ ]:
# the shop_df has cities as a first word. 
#In Russian, geography has big value and give us more accurate prediction
full_name = shops_df['shop_name'].str.split(' ', n=0, expand=True)
cities = full_name[0].str.split('.', n=0, expand=True)
cities = cities[0]
cities = cities.replace(['!Якутск'], 'Якутск')
# we need to label_encode the cities
label_encoder = LabelEncoder()
cities_encoded = label_encoder.fit_transform(cities)
shops_df['city_id'] = pd.Series(cities_encoded, index=shops_df.index)
# joining the 'cities_id' column to the sales_train_df
sales_train_df = sales_train_df.join(
                  shops_df[['shop_id', 'city_id']].set_index('shop_id'), on='shop_id'
                 )
# the result of ths - classification sales by geography
sales_train_df

In [ ]:
# grouping cities
grouped_city_names = cities.groupby(cities, as_index=True).count()
grouped_city_by_count = sales_train_df[['city_id', 'item_id']].groupby(sales_train_df['city_id'],
                                               as_index=True).count()
#plotting the histogram for cities
plot_plt_histogram(sales_train_df['city_id'], bins=cities_encoded.shape[0], grid=True, 
                   need_ylim=True, ylim=200000)
#printing the result
print('median: ',grouped_city_by_count.median()[0],
      'min: ', grouped_city_by_count.min()[0], 'max: ', grouped_city_by_count.max()[0],
      "std: ", grouped_city_by_count.std()[0], 'mean: ', grouped_city_by_count.mean()[0])
grouped_city_by_count

Let's watch on items and what's their distribution

In [ ]:
# groupping the item by frequency
grouped_item_id = sales_train_df['item_id'].groupby(sales_train_df['item_id'],
                                  as_index=True).count()

#printing the histogram of 
plot_plt_histogram(sales_train_df['item_id'], bins=items_df.shape[0],
                   height=25, width=100,
                   need_ylim=True, ylim=1000)
# The result of this histogram is frequency of changing the price of item. 
# The high limit for y is 1000 because if it is without limit the plot will be unreadble
print(grouped_item_id)
print(grouped_item_id.sort_values(ascending=False))

In [ ]:
# we need to group our frequency
plot_plt_histogram(grouped_item_id, bins=grouped_item_id.shape[0],
                   height=40, width=100,
                   need_ylim=True, ylim=800, need_xlim=True, xlim=1000)
# printing the median of times of changing the price
print('median:',grouped_item_id.median(), 'mode:',grouped_item_id.mode()[0],
      'min:', grouped_item_id.min(), 'max:', grouped_item_id.max(),
      "std:", grouped_item_id.std(), 'mean:', grouped_item_id.mean())
grouped_item_id

We can see, that half of items has more than 32 times of changing the price and half of items has less. So many items has no relationship to time.
So, we can classificate items to groups as 'rare', 'often', 'popular' as 0, 1, 2. 'Rare' is from 1 to 12, 'often' is from 13 to 365, 'popular' is from 366 to max.

In [ ]:
# for working with grouped data we need to copy variable grouped_item_id
freq = pd.Series(grouped_item_id.copy().to_numpy())
# create a dataframe for table of classification 
classification_by_freq = pd.DataFrame()
classification_by_freq['item_id'] = freq.index
classification_by_freq['num_freq'] = freq
# 'freq' column is our classifier
classification_by_freq['freq'] = 0
classification_by_freq['freq'][(classification_by_freq['num_freq'] >= 13) \
                               & (classification_by_freq['num_freq'] <=365)] = 1
classification_by_freq['freq'][(classification_by_freq['num_freq'] >= 366) \
                               & (classification_by_freq['num_freq'] <= classification_by_freq['num_freq'].max())] = 2
# if I need to cut on more parts, write code to HERE \/

# printing the count of every class
print('count items:', 
      classification_by_freq['freq'].groupby(classification_by_freq['freq'], as_index=True).count())

In [ ]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)
item_id_grouped_by_mean = sales_train_df[['item_id', 'item_price']].groupby(sales_train_df['item_id'],
                                  as_index=True).mean()
item_id_grouped_by_median = sales_train_df[['item_id', 'item_price']].groupby(sales_train_df['item_id'],
                                  as_index=True).median()
#print(item_id_grouped_by_mean.sort_values(by='item_price', ascending=False))
plot_plt_scatter(item_id_grouped_by_mean['item_id'], item_id_grouped_by_mean['item_price'])
plot_plt_scatter(item_id_grouped_by_median['item_id'], item_id_grouped_by_median['item_price'])

In [ ]:
time_ser = sales_train_df[['date', 'item_price']][sales_train_df['item_id'] == 20949]
#time_ser
grouped_time_series = time_ser.groupby('date', as_index=False).mean()
grouped_time_series.plot(x='date', y='item_price', figsize=(100, 30))
grouped_time_series
print()